In [7]:
import pandas as pd

In [ ]:
reviews = pd.read_json('D:/Downloads/reviews_1500k.json', lines=True)

In [ ]:
reviews

In [ ]:
# Note that this takes a while to run
reviews['text'] = reviews['text'].str.lower().str.split()

### Removing stop words

In [ ]:
# Download set of stop words
import nltk
nltk.download('stopwords')
stop_words_set = set(stopwords.words('english'))

In [ ]:
# Removing stop words
reviews['text'] = reviews['text'].apply(lambda x: [item for item in x if item not in stop_words_set])

In [ ]:
' '.join(reviews['text'][0])

In [45]:
# Convert back to string, takes a while to run
reviews['text'].apply(lambda x: [' '.join(x)])

MemoryError: 